# 1 SPE（Sinusoidal Positional Encoding，正弦余弦位置编码）

$$
\text{PE}(pos, 2i) = \sin\left(\frac{pos}{10000^{\frac{2i}{\text{embed\_dim}}}} \right)
$$
$$
\text{PE}(pos, 2i + 1) = \cos\left(\frac{pos}{10000^{\frac{2i}{\text{embed\_dim}}}} \right)
$$

简化一下，式子中的div_term其实就是：
$$
10000^{-\frac{2i}{\text{embed\_dim}}} = \frac{1}{10000^{\frac{2i}{\text{embed\_dim}}}}
$$


In [31]:
import torch
import math
import torch.nn as nn


class PositionalEncoding(nn.Module):
    def __init__(self, embed_dim=16, max_len=1000):
        super().__init__()

        self.pe = torch.zeros(max_len, embed_dim)
        self.base = 10000
        # 从0到最大位置的序号
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        # 计算式子中的分母（可以从上面公式中看出，奇数偶数位置的计算都是相同的分母）
        div_term = torch.exp(torch.arange(0, embed_dim, 2, dtype=torch.float) * -(math.log(self.base) / embed_dim))
        # 根据公式填充位置编码矩阵
        self.pe[:, 0::2] = torch.sin(position * div_term)  # 偶数位置使用sin
        self.pe[:, 1::2] = torch.cos(position * div_term)  # 奇数位置使用cos
        # 添加batch维度
        self.pe = self.pe.unsqueeze(0)

    def forward(self, x):
        _, seq_len, _ = x.shape
        # 为x每个位置添加位置编码
        return x + self.pe[:, :seq_len]


## 测试

In [35]:
pe = PositionalEncoding(embed_dim=4)
x = torch.randn(1, 2, 4)
print("原始输入：")
print(x)
print("带位置编码：")
pe.forward(x)

原始输入：
tensor([[[-0.3361, -0.1531, -0.0838, -1.1054],
         [ 0.2267, -0.0934,  0.2530,  1.2269]]])
带位置编码：


tensor([[[-0.3361,  0.8469, -0.0838, -0.1054],
         [ 1.0681,  0.4469,  0.2630,  2.2269]]])